In [23]:
# ensure installed packages can be used in the current notebook within the context of the current Python kernel
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install zipfile
!{sys.executable} -m pip install io
!{sys.executable} -m pip install lxml

ERROR: Could not find a version that satisfies the requirement zipfile (from versions: none)
ERROR: No matching distribution found for zipfile
ERROR: Could not find a version that satisfies the requirement io (from versions: none)
ERROR: No matching distribution found for io


In [24]:
import requests
import pandas as pd
import re
import zipfile
import requests
import warnings
from io import BytesIO
from lxml import html

warnings.filterwarnings("ignore", message="Unverified HTTPS request")

In [25]:
# get state geography data from the U.S. Census Bureau

# read state file csv and create dataframe
all_census_states = pd.read_csv(
    'https://www2.census.gov/geo/docs/reference/state.txt',
    delimiter='|',
    header=0,
    usecols=[0,1,2],
    names=['state_fips', 'state_abbr', 'c_state_name'],
    dtype=str
)

# define a list of FIPS codes that includes Alaska, D.C., and the internally-autonomous entities the U.S. has some control over
suzerainty_fips = ['02','11','60','66','69','72','74','78']

# filter out Alaska and these Suzerainty entities
census_states = all_census_states[~all_census_states.state_fips.isin(suzerainty_fips)]

print(census_states.shape, '\n', census_states.dtypes)
census_states.head(100)

(49, 3) 
 state_fips      object
state_abbr      object
c_state_name    object
dtype: object


state_fips state_abbr    c_state_name
0          01         AL         Alabama
2          04         AZ         Arizona
3          05         AR        Arkansas
4          06         CA      California
5          08         CO        Colorado
6          09         CT     Connecticut
7          10         DE        Delaware
9          12         FL         Florida
10         13         GA         Georgia
11         15         HI          Hawaii
12         16         ID           Idaho
13         17         IL        Illinois
14         18         IN         Indiana
15         19         IA            Iowa
16         20         KS          Kansas
17         21         KY        Kentucky
18         22         LA       Louisiana
19         23         ME           Maine
20         24         MD        Maryland
21         25         MA   Massachusetts
22         26         MI        Michigan
23         27         MN       Minnesota
24         28         MS     Mississippi
25         29         MO        Missouri
26         30         MT         Montana
27         31         NE        Nebraska
28         32         NV          Nevada
29         33         NH   New Hampshire
30         34         NJ      New Jersey
31         35         NM      New Mexico
32         36         NY        New York
33         37         NC  North Carolina
34         38         ND    North Dakota
35         39         OH            Ohio
36         40         OK        Oklahoma
37         41         OR          Oregon
38         42         PA    Pennsylvania
39         44         RI    Rhode Island
40         45         SC  South Carolina
41         46         SD    South Dakota
42         47         TN       Tennessee
43         48         TX           Texas
44         49         UT            Utah
45         50         VT         Vermont
46         51         VA        Virginia
47         53         WA      Washington
48         54         WV   West Virginia
49         55         WI       Wisconsin
50         56         WY         Wyoming

In [26]:
# enter base year for the state upper and lower legislative house and county gazetteer files
census_year = 2024

In [27]:
# get county geography data from the U.S. Census Bureau Gazetteer files

# base_url where all the county gazetteer files live
gazetteer_url = f"https://www2.census.gov/geo/docs/maps-data/data/gazetteer/{census_year}_Gazetteer/"

# make an http request for the page
page = requests.request(
    method='GET', 
    url=gazetteer_url,
    headers={ "Accept": "application/json" }
)

# parse the page and return a DOM tree
tree = html.fromstring(page.content)

# use XPath to return a list of link texts ('a' elements within the 'table' element) from the DOM
gazatteer_files = tree.xpath('//td/a/text()')

# filter the list to return only county file names
county_files = [c for c in gazatteer_files if re.match(r'.*counties.*\.txt', c)]

# Convert the list to a series
county_files_series = pd.Series(county_files, name="county_file_name")

# Convert series to dataframe
census_gaz_counties = county_files_series.to_frame()

# filter out Alaska and these Suzerainty entities
census_gaz_counties = census_gaz_counties[~census_gaz_counties.county_file_name.str.extract(r'_(\d{2})\.txt')[0].isin(suzerainty_fips)]

print(census_gaz_counties.shape, '\n', census_gaz_counties.dtypes)
census_gaz_counties.head(100)


(49, 1) 
 county_file_name    object
dtype: object


county_file_name
0   2024_gaz_counties_01.txt
2   2024_gaz_counties_04.txt
3   2024_gaz_counties_05.txt
4   2024_gaz_counties_06.txt
5   2024_gaz_counties_08.txt
6   2024_gaz_counties_09.txt
7   2024_gaz_counties_10.txt
9   2024_gaz_counties_12.txt
10  2024_gaz_counties_13.txt
11  2024_gaz_counties_15.txt
12  2024_gaz_counties_16.txt
13  2024_gaz_counties_17.txt
14  2024_gaz_counties_18.txt
15  2024_gaz_counties_19.txt
16  2024_gaz_counties_20.txt
17  2024_gaz_counties_21.txt
18  2024_gaz_counties_22.txt
19  2024_gaz_counties_23.txt
20  2024_gaz_counties_24.txt
21  2024_gaz_counties_25.txt
22  2024_gaz_counties_26.txt
23  2024_gaz_counties_27.txt
24  2024_gaz_counties_28.txt
25  2024_gaz_counties_29.txt
26  2024_gaz_counties_30.txt
27  2024_gaz_counties_31.txt
28  2024_gaz_counties_32.txt
29  2024_gaz_counties_33.txt
30  2024_gaz_counties_34.txt
31  2024_gaz_counties_35.txt
32  2024_gaz_counties_36.txt
33  2024_gaz_counties_37.txt
34  2024_gaz_counties_38.txt
35  2024_gaz_counties_39.txt
36  2024_gaz_counties_40.txt
37  2024_gaz_counties_41.txt
38  2024_gaz_counties_42.txt
39  2024_gaz_counties_44.txt
40  2024_gaz_counties_45.txt
41  2024_gaz_counties_46.txt
42  2024_gaz_counties_47.txt
43  2024_gaz_counties_48.txt
44  2024_gaz_counties_49.txt
45  2024_gaz_counties_50.txt
46  2024_gaz_counties_51.txt
47  2024_gaz_counties_53.txt
48  2024_gaz_counties_54.txt
49  2024_gaz_counties_55.txt
50  2024_gaz_counties_56.txt

In [28]:
# parse county geography data from the U.S. Census Bureau Gazetteer files

# read each county file csv
census_county_files = [
    pd.read_csv(
        gazetteer_url + county_file_name,
        delimiter='\t',
        lineterminator='\n',
        header=0,
        usecols=[1,3],
        names=['geoid', 'c_county_name'],
        dtype=str
    ) for county_file_name in census_gaz_counties['county_file_name']]

# combine into a dataframe
census_counties = pd.concat(census_county_files, ignore_index=True)

# create state FIPS codes from the 5-digit 'geoid'
census_counties['state_fips'] = census_counties['geoid'].str[:2]

print(census_counties.shape, '\n', census_counties.dtypes)
census_counties.head(100)

(3113, 3) 
 geoid            object
c_county_name    object
state_fips       object
dtype: object


geoid      c_county_name state_fips
0   01001     Autauga County         01
1   01003     Baldwin County         01
2   01005     Barbour County         01
3   01007        Bibb County         01
4   01009      Blount County         01
..    ...                ...        ...
95  05027    Columbia County         05
96  05029      Conway County         05
97  05031   Craighead County         05
98  05033    Crawford County         05
99  05035  Crittenden County         05

[100 rows x 3 columns]

In [29]:
# merge Census counties and states
# Perform a left join on 'state_fips'
county_state_df = census_counties.merge(
    census_states[['state_fips', 'c_state_name']], 
    on='state_fips', 
    how='left'
)

# Drop the 'state_fips' column
census_geographies = county_state_df.drop(columns=['state_fips'], axis=1)

print(census_geographies.shape, '\n', census_geographies.dtypes)
census_geographies.head(100)

(3113, 3) 
 geoid            object
c_county_name    object
c_state_name     object
dtype: object


geoid      c_county_name c_state_name
0   01001     Autauga County      Alabama
1   01003     Baldwin County      Alabama
2   01005     Barbour County      Alabama
3   01007        Bibb County      Alabama
4   01009      Blount County      Alabama
..    ...                ...          ...
95  05027    Columbia County     Arkansas
96  05029      Conway County     Arkansas
97  05031   Craighead County     Arkansas
98  05033    Crawford County     Arkansas
99  05035  Crittenden County     Arkansas

[100 rows x 3 columns]

In [30]:
# Helper function to format state names for URLs
def format_state(state):
    return state.replace(".", "").replace(",", "").replace(" ", "-").lower()

# Function to get state results from the New York Times
def get_nyt_state_results(state):
    # Request JSON data for the specified formatted state
    formatted_state = format_state(state)
    url = f"https://static01.nyt.com/elections-assets/pages/data/2024-11-05/results-{formatted_state}-president.json"
    response = requests.get(url)
    data = response.json()
    
    # Initialize dataframe and extract races
    races = data.get("races", [])
    rows = []
    
    for race in races:
        for unit in race.get("reporting_units", []):
            if unit.get("fips_county") is not None:
                for candidate in unit.get("candidates", []):
                    row = {
                        "fips_state": unit.get("fips_state"),
                        "fips_county": unit.get("fips_county"),
                        "total_votes": unit.get("total_votes"),
                        "nyt_id": candidate.get("nyt_id"),
                        "total": candidate.get("votes", {}).get("total", 0)
                    }
                    rows.append(row)
    
    # Convert rows to a dataframe
    df = pd.DataFrame(rows)
    
    # Aggregate some states' data to county level
    if formatted_state in ["connecticut","vermont","maine","massachusetts","new-hampshire","rhode-island","washington-dc"]:
        df = df.groupby(["nyt_id", "fips_state", "fips_county"], as_index=False).agg({
            "total": "sum",
            "total_votes": "sum"
        })
    
    # Pivot the data so each candidate's votes are in their own column
    df_pivoted = df.pivot_table(index=["fips_state", "fips_county", "total_votes"], columns="nyt_id", values="total", fill_value=0)
    df_pivoted.reset_index(inplace=True)
    
    # Combine fips_state and fips_county columns to create county_fips
    df_pivoted["county_fips"] = df_pivoted["fips_state"].astype(str) + df_pivoted["fips_county"].astype(str)
    
    return df_pivoted

In [31]:
# Function to get state results from Fox News
def get_fox_state_results(state_abbr):
    # Request JSON data for the specified state
    url = f"https://feeds-elections.foxnews.com/archive/politics/elections/2024/3/2024_Generals/President/{state_abbr}/county_level_results/file.json"
    response = requests.get(url)
    data = response.json()

    # Process lookup table
    lookup_counties = data['lookup']['counties']
    # Convert JSON to dataframe
    lookup_df = pd.DataFrame.from_dict(lookup_counties, orient='index').reset_index(drop=False)
    # Drop the "number" and "precinctsReporting" columns
    lookup_df = lookup_df.drop(columns=["name", "number", "precinctsReporting"])
    lookup_df = lookup_df.rename(columns={'index': 'countyId', 'fipsCode': 'county_fips'})
    # Convert countyId to integer and fipsCode to string
    lookup_df['countyId'] = lookup_df['countyId'].astype(int)
    lookup_df['county_fips'] = lookup_df['county_fips'].astype(str)
    # pad with leading zeros
    lookup_df['county_fips'] = lookup_df['county_fips'].str.zfill(5)

    # Process results table
    results = data['results']
    # Flatten the JSON
    rows = []
    for result in results:
        party_name = result['candidate']['partyName']
        for vote_data in result['countyVotes']:
            county_id = vote_data['countyId']
            vote_count = vote_data['votes']['count']
            row = {
                 'partyName': party_name, 
                 'countyId': county_id, 
                 'count': vote_count
            }
            rows.append(row)

    # Create the dataframe
    cadidate_df = pd.DataFrame(rows)
    
    # Group by county and calculate total votes
    grouped_by_county = cadidate_df.groupby('countyId', as_index=False)['count'].sum().rename(columns={'count': 'total_votes'})
    
    # Filter results for Democrat and Republican candidates only
    filtered_df = cadidate_df[cadidate_df['partyName'].isin(['Democrat', 'Republican'])]
    pivoted_df = filtered_df.pivot(index='countyId', columns='partyName', values='count').reset_index().fillna(0)
    pivoted_df = pivoted_df.rename(columns={'Democrat': 'votes_dem', 'Republican': 'votes_gop'})

    # Merge total votes and pivoted results
    results_df = pivoted_df.merge(grouped_by_county, on='countyId', how='left')

    # Merge with lookup to get FIPS codes
    results_lookup_df = results_df.merge(lookup_df, on='countyId', how='left')

    # Reorder columns
    results_lookup_df = results_lookup_df[['county_fips', 'total_votes', 'votes_dem', 'votes_gop']]
    
    return results_lookup_df

In [32]:
# get election results data from the New York Times

# Define candidates and fields
two_party_candidates = ["harris-k", "trump-d"]
fields = ["county_fips", "total_votes"] + two_party_candidates

# Define state names
state_names = ["Washington, D.C." if state == "District of Columbia" else state for state in census_states['c_state_name']]

# Retrieve results for each state
contiguous_nyt_state_results = [get_nyt_state_results(state) for state in state_names]

# Combine all state dataframes
contiguous_nyt_election_results = pd.concat(contiguous_nyt_state_results, ignore_index=True)

# Reorder columns and rename
contiguous_nyt_election_results = contiguous_nyt_election_results[["county_fips", "total_votes"] + two_party_candidates]
contiguous_nyt_election_results = contiguous_nyt_election_results.rename(columns={"harris-k": "votes_dem", "trump-d": "votes_gop"})

# Calculate GOP percentage
contiguous_nyt_election_results["per_gop"] = contiguous_nyt_election_results["votes_gop"] / contiguous_nyt_election_results["total_votes"]

# Calculate DEM percentage
contiguous_nyt_election_results["per_dem"] = contiguous_nyt_election_results["votes_dem"] / contiguous_nyt_election_results["total_votes"]

# Calculate vote difference
contiguous_nyt_election_results["diff"] = contiguous_nyt_election_results["votes_gop"] - contiguous_nyt_election_results["votes_dem"]

# Calculate percentage point difference
contiguous_nyt_election_results["per_point_diff"] = contiguous_nyt_election_results["per_gop"] - contiguous_nyt_election_results["per_dem"]

# Ensure column types are correctly set
contiguous_nyt_election_results["county_fips"] = contiguous_nyt_election_results["county_fips"].astype(str)
contiguous_nyt_election_results["total_votes"] = contiguous_nyt_election_results["total_votes"].astype("Int64")
contiguous_nyt_election_results["votes_dem"] = contiguous_nyt_election_results["votes_dem"].astype("Int64")
contiguous_nyt_election_results["votes_gop"] = contiguous_nyt_election_results["votes_gop"].astype("Int64")

# Sort by county_fips
contiguous_nyt_election_results = contiguous_nyt_election_results.sort_values(by="county_fips").reset_index(drop=True)

print(contiguous_nyt_election_results.shape, '\n', contiguous_nyt_election_results.dtypes)
contiguous_nyt_election_results.head(100)

(3111, 8) 
 nyt_id
county_fips        object
total_votes         Int64
votes_dem           Int64
votes_gop           Int64
per_gop           float64
per_dem           float64
diff              float64
per_point_diff    float64
dtype: object


nyt_id county_fips  total_votes  votes_dem  votes_gop   per_gop   per_dem  \
0            01001        28190       7439      20484  0.726641  0.263888   
1            01003       121808      24934      95798  0.786467  0.204699   
2            01005         9832       4158       5606  0.570179  0.422905   
3            01007         9241       1619       7572  0.819392  0.175197   
4            01009        28115       2576      25354  0.901796  0.091624   
..             ...          ...        ...        ...       ...       ...   
95           05027         7940       2466       5367  0.675945  0.310579   
96           05029         8541       2449       5893  0.689966  0.286735   
97           05031        37242      11210      25152  0.675367  0.301004   
98           05033        23834       4753      18615  0.781027  0.199421   
99           05035        14681       7362       7028  0.478714  0.501464   

nyt_id     diff  per_point_diff  
0       13045.0        0.462753  
1       70864.0        0.581768  
2        1448.0        0.147274  
3        5953.0        0.644194  
4       22778.0        0.810173  
..          ...             ...  
95       2901.0        0.365365  
96       3444.0        0.403231  
97      13942.0        0.374362  
98      13862.0        0.581606  
99       -334.0       -0.022750  

[100 rows x 8 columns]

In [33]:
# get election results data from Fox News

# Define state abbreviations
state_abbrs = [state for state in census_states['state_abbr']]

# Retrieve results for each state
contiguous_fox_state_results = [get_fox_state_results(state_abbr) for state_abbr in state_abbrs]

# Combine all state dataframes
contiguous_fox_election_results = pd.concat(contiguous_fox_state_results, ignore_index=True)

# Calculate GOP percentage
contiguous_fox_election_results["per_gop"] = contiguous_fox_election_results["votes_gop"] / contiguous_fox_election_results["total_votes"]

# Calculate DEM percentage
contiguous_fox_election_results["per_dem"] = contiguous_fox_election_results["votes_dem"] / contiguous_fox_election_results["total_votes"]

# Calculate vote difference
contiguous_fox_election_results["diff"] = contiguous_fox_election_results["votes_gop"] - contiguous_fox_election_results["votes_dem"]

# Calculate percentage point difference
contiguous_fox_election_results["per_point_diff"] = contiguous_fox_election_results["per_gop"] - contiguous_fox_election_results["per_dem"]

# Ensure column types are correctly set
contiguous_fox_election_results["county_fips"] = contiguous_fox_election_results["county_fips"].astype(str)
contiguous_fox_election_results["total_votes"] = contiguous_fox_election_results["total_votes"].astype("Int64")
contiguous_fox_election_results["votes_dem"] = contiguous_fox_election_results["votes_dem"].astype("Int64")
contiguous_fox_election_results["votes_gop"] = contiguous_fox_election_results["votes_gop"].astype("Int64")

# Sort by county_fips
contiguous_fox_election_results = contiguous_fox_election_results.sort_values(by="county_fips").reset_index(drop=True)

print(contiguous_fox_election_results.shape, '\n', contiguous_fox_election_results.dtypes)
contiguous_fox_election_results.head(100)

(3112, 8) 
 county_fips        object
total_votes         Int64
votes_dem           Int64
votes_gop           Int64
per_gop           float64
per_dem           float64
diff                int64
per_point_diff    float64
dtype: object


county_fips  total_votes  votes_dem  votes_gop   per_gop   per_dem   diff  \
0        01001        28190       7439      20484  0.726641  0.263888  13045   
1        01003       121808      24934      95798  0.786467  0.204699  70864   
2        01005         9832       4158       5606  0.570179  0.422905   1448   
3        01007         9241       1619       7572  0.819392  0.175197   5953   
4        01009        28115       2576      25354  0.901796  0.091624  22778   
..         ...          ...        ...        ...       ...       ...    ...   
95       05027         7940       2466       5367  0.675945  0.310579   2901   
96       05029         8541       2449       5893  0.689966  0.286735   3444   
97       05031        37242      11210      25152  0.675367  0.301004  13942   
98       05033        23834       4753      18615  0.781027  0.199421  13862   
99       05035        14681       7362       7028  0.478714  0.501464   -334   

    per_point_diff  
0         0.462753  
1         0.581768  
2         0.147274  
3         0.644194  
4         0.810173  
..             ...  
95        0.365365  
96        0.403231  
97        0.374362  
98        0.581606  
99       -0.022750  

[100 rows x 8 columns]

In [34]:
# merge election results on to the Census geography data

# election_results should contain: 'county_fips', 'votes_gop', 'votes_dem', 'total_votes'
# census_geographies should contain: 'geoid', 'c_county_name', 'c_state_name'

# Perform a left join on <source>_election_results and census_geographies
results_geographies = pd.merge(
    contiguous_fox_election_results,
    census_geographies,
    left_on="county_fips",
    right_on="geoid",
    how="left"
)

# Rename columns from the joined data
results_geographies.rename(columns={"c_state_name": "state_name", "c_county_name": "county_name"}, inplace=True)

# Reorder columns
election_results = results_geographies[[
    "state_name", "county_fips", "county_name", "votes_gop", "votes_dem", 
    "total_votes", "diff", "per_gop", "per_dem", "per_point_diff"
]]

print(election_results.shape, '\n', election_results.dtypes)
election_results.head(100)

(3112, 10) 
 state_name         object
county_fips        object
county_name        object
votes_gop           Int64
votes_dem           Int64
total_votes         Int64
diff                int64
per_gop           float64
per_dem           float64
per_point_diff    float64
dtype: object


state_name county_fips        county_name  votes_gop  votes_dem  \
0     Alabama       01001     Autauga County      20484       7439   
1     Alabama       01003     Baldwin County      95798      24934   
2     Alabama       01005     Barbour County       5606       4158   
3     Alabama       01007        Bibb County       7572       1619   
4     Alabama       01009      Blount County      25354       2576   
..        ...         ...                ...        ...        ...   
95   Arkansas       05027    Columbia County       5367       2466   
96   Arkansas       05029      Conway County       5893       2449   
97   Arkansas       05031   Craighead County      25152      11210   
98   Arkansas       05033    Crawford County      18615       4753   
99   Arkansas       05035  Crittenden County       7028       7362   

    total_votes   diff   per_gop   per_dem  per_point_diff  
0         28190  13045  0.726641  0.263888        0.462753  
1        121808  70864  0.786467  0.204699        0.581768  
2          9832   1448  0.570179  0.422905        0.147274  
3          9241   5953  0.819392  0.175197        0.644194  
4         28115  22778  0.901796  0.091624        0.810173  
..          ...    ...       ...       ...             ...  
95         7940   2901  0.675945  0.310579        0.365365  
96         8541   3444  0.689966  0.286735        0.403231  
97        37242  13942  0.675367  0.301004        0.374362  
98        23834  13862  0.781027  0.199421        0.581606  
99        14681   -334  0.478714  0.501464       -0.022750  

[100 rows x 10 columns]

In [35]:
# get Washington, D.C. legislative upper house geography data from the U.S. Census Bureau Gazetteer national zip file

# base_url where the national state legislative upper house gazetteer file lives
national_gazetteer_sldu_url = f"https://www2.census.gov/geo/docs/maps-data/data/gazetteer/{census_year}_Gazetteer/{census_year}_Gaz_sldu_national.zip"

# download the zip file content into memory
sldu_response = requests.get(national_gazetteer_sldu_url)
national_gazetteer_sldu_zip = BytesIO(sldu_response.content)  # Use BytesIO to hold the downloaded content in memory

# open the zip file from the in-memory content
with zipfile.ZipFile(national_gazetteer_sldu_zip, 'r') as national_gazetteer_sldu_zip_ref:
    # List files inside the zip
    unzipped_sldu_files = national_gazetteer_sldu_zip_ref.namelist()

    # find the tab-delimited text file (assuming it's a .txt file)
    tab_delimited_sldu_file = None
    for sldu_file in unzipped_sldu_files:
        if sldu_file.endswith(".txt"):
            tab_delimited_sldu_file = sldu_file
            break

    if not tab_delimited_sldu_file:
        print("No tab-delimited text file found.")
        exit(1)

    # read the tab-delimited text file directly into a Pandas DataFrame
    with national_gazetteer_sldu_zip_ref.open(tab_delimited_sldu_file) as sldu_file:
        national_sldusts = pd.read_csv(
            sldu_file,
            delimiter='\t',
            header=0,
            usecols=[1,2],
            names=['geoid', 'c_sldust_name'],
            dtype=str
        )
        
print(national_sldusts.shape, '\n', national_sldusts.dtypes)
national_sldusts.head(100)

(1964, 2) 
 geoid            object
c_sldust_name    object
dtype: object


geoid             c_sldust_name
0   01001   State Senate District 1
1   01002   State Senate District 2
2   01003   State Senate District 3
3   01004   State Senate District 4
4   01005   State Senate District 5
..    ...                       ...
95  05011  State Senate District 11
96  05012  State Senate District 12
97  05013  State Senate District 13
98  05014  State Senate District 14
99  05015  State Senate District 15

[100 rows x 2 columns]

In [36]:
# get Alaska legislative lower house geography data from the U.S. Census Bureau Gazetteer national zip file

# base_url where the national state legislative lower house gazetteer file lives
national_gazetteer_sldl_url = f"https://www2.census.gov/geo/docs/maps-data/data/gazetteer/{census_year}_Gazetteer/{census_year}_Gaz_sldl_national.zip"

# download the zip file content into memory
sldl_response = requests.get(national_gazetteer_sldl_url)
national_gazetteer_sldl_zip = BytesIO(sldl_response.content)  # Use BytesIO to hold the downloaded content in memory

# open the zip file from the in-memory content
with zipfile.ZipFile(national_gazetteer_sldl_zip, 'r') as national_gazetteer_sldl_zip_ref:
    # List files inside the zip
    unzipped_sldl_files = national_gazetteer_sldl_zip_ref.namelist()

    # find the tab-delimited text file (assuming it's a .txt file)
    tab_delimited_sldl_file = None
    for sldl_file in unzipped_sldl_files:
        if sldl_file.endswith(".txt"):
            tab_delimited_sldl_file = sldl_file
            break

    if not tab_delimited_sldl_file:
        print("No tab-delimited text file found.")
        exit(1)

    # read the tab-delimited text file directly into a Pandas DataFrame
    with national_gazetteer_sldl_zip_ref.open(tab_delimited_sldl_file) as sldl_file:
        national_sldlsts = pd.read_csv(
            sldl_file,
            delimiter='\t',
            header=0,
            usecols=[1,2],
            names=['geoid', 'c_sldlst_name'],
            dtype=str
        )
        
print(national_sldlsts.shape, '\n', national_sldlsts.dtypes)
national_sldlsts.head(100)

(4879, 2) 
 geoid            object
c_sldlst_name    object
dtype: object


geoid             c_sldlst_name
0   01001    State House District 1
1   01002    State House District 2
2   01003    State House District 3
3   01004    State House District 4
4   01005    State House District 5
..    ...                       ...
95  01096   State House District 96
96  01097   State House District 97
97  01098   State House District 98
98  01099   State House District 99
99  01100  State House District 100

[100 rows x 2 columns]

In [37]:
# get DC state geography data from the U.S. Census Bureau

# filter the national sldusts dataframe for Washington, D.C.
dc_geographies = national_sldusts[national_sldusts.geoid.str.startswith("11")].reset_index(drop=True)

# add Alaska name to state_name column
dc_geographies["c_state_name"] ="District of Columbia"

# Reorder columns
dc_geographies = dc_geographies[["c_state_name", "geoid", "c_sldust_name"]]

print(dc_geographies.shape, '\n', dc_geographies.dtypes)
dc_geographies.head(100)

(8, 3) 
 c_state_name     object
geoid            object
c_sldust_name    object
dtype: object


c_state_name  geoid c_sldust_name
0  District of Columbia  11001        Ward 1
1  District of Columbia  11002        Ward 2
2  District of Columbia  11003        Ward 3
3  District of Columbia  11004        Ward 4
4  District of Columbia  11005        Ward 5
5  District of Columbia  11006        Ward 6
6  District of Columbia  11007        Ward 7
7  District of Columbia  11008        Ward 8

In [38]:
# get Alaska state geography data from the U.S. Census Bureau

# filter the national sldlsts dataframe for Alaska
ak_geographies = national_sldlsts[national_sldlsts.geoid.str.startswith("02")].reset_index(drop=True)

# add Alaska name to state_name column
ak_geographies["c_state_name"] ="Alaska"

# Reorder columns
ak_geographies = ak_geographies[["c_state_name", "geoid", "c_sldlst_name"]]

print(ak_geographies.shape, '\n', ak_geographies.dtypes)
ak_geographies.head(100)

(40, 3) 
 c_state_name     object
geoid            object
c_sldlst_name    object
dtype: object


c_state_name  geoid            c_sldlst_name
0        Alaska  02001   State House District 1
1        Alaska  02002   State House District 2
2        Alaska  02003   State House District 3
3        Alaska  02004   State House District 4
4        Alaska  02005   State House District 5
5        Alaska  02006   State House District 6
6        Alaska  02007   State House District 7
7        Alaska  02008   State House District 8
8        Alaska  02009   State House District 9
9        Alaska  02010  State House District 10
10       Alaska  02011  State House District 11
11       Alaska  02012  State House District 12
12       Alaska  02013  State House District 13
13       Alaska  02014  State House District 14
14       Alaska  02015  State House District 15
15       Alaska  02016  State House District 16
16       Alaska  02017  State House District 17
17       Alaska  02018  State House District 18
18       Alaska  02019  State House District 19
19       Alaska  02020  State House District 20
20       Alaska  02021  State House District 21
21       Alaska  02022  State House District 22
22       Alaska  02023  State House District 23
23       Alaska  02024  State House District 24
24       Alaska  02025  State House District 25
25       Alaska  02026  State House District 26
26       Alaska  02027  State House District 27
27       Alaska  02028  State House District 28
28       Alaska  02029  State House District 29
29       Alaska  02030  State House District 30
30       Alaska  02031  State House District 31
31       Alaska  02032  State House District 32
32       Alaska  02033  State House District 33
33       Alaska  02034  State House District 34
34       Alaska  02035  State House District 35
35       Alaska  02036  State House District 36
36       Alaska  02037  State House District 37
37       Alaska  02038  State House District 38
38       Alaska  02039  State House District 39
39       Alaska  02040  State House District 40

In [39]:
# combine D.C. legislative upper house geography data with Alaska legislative lower house geography data

# rename the columns in dc_geographies
dc_geographies.rename(columns={"c_sldust_name": "c_county_name"}, inplace=True)

# rename the columns in ak_geographies
ak_geographies.rename(columns={"c_sldlst_name": "c_county_name"}, inplace=True)


ak_dc_geographies = pd.concat([ak_geographies, dc_geographies], ignore_index=True)

#sort by geoid
ak_dc_geographies = ak_dc_geographies.sort_values(by="geoid").reset_index(drop=True)

# print the shape and data types of the combined dataframe
print(ak_dc_geographies.shape, '\n', ak_dc_geographies.dtypes)
ak_dc_geographies.head(100)

(48, 3) 
 c_state_name     object
geoid            object
c_county_name    object
dtype: object


c_state_name  geoid            c_county_name
0                 Alaska  02001   State House District 1
1                 Alaska  02002   State House District 2
2                 Alaska  02003   State House District 3
3                 Alaska  02004   State House District 4
4                 Alaska  02005   State House District 5
5                 Alaska  02006   State House District 6
6                 Alaska  02007   State House District 7
7                 Alaska  02008   State House District 8
8                 Alaska  02009   State House District 9
9                 Alaska  02010  State House District 10
10                Alaska  02011  State House District 11
11                Alaska  02012  State House District 12
12                Alaska  02013  State House District 13
13                Alaska  02014  State House District 14
14                Alaska  02015  State House District 15
15                Alaska  02016  State House District 16
16                Alaska  02017  State House District 17
17                Alaska  02018  State House District 18
18                Alaska  02019  State House District 19
19                Alaska  02020  State House District 20
20                Alaska  02021  State House District 21
21                Alaska  02022  State House District 22
22                Alaska  02023  State House District 23
23                Alaska  02024  State House District 24
24                Alaska  02025  State House District 25
25                Alaska  02026  State House District 26
26                Alaska  02027  State House District 27
27                Alaska  02028  State House District 28
28                Alaska  02029  State House District 29
29                Alaska  02030  State House District 30
30                Alaska  02031  State House District 31
31                Alaska  02032  State House District 32
32                Alaska  02033  State House District 33
33                Alaska  02034  State House District 34
34                Alaska  02035  State House District 35
35                Alaska  02036  State House District 36
36                Alaska  02037  State House District 37
37                Alaska  02038  State House District 38
38                Alaska  02039  State House District 39
39                Alaska  02040  State House District 40
40  District of Columbia  11001                   Ward 1
41  District of Columbia  11002                   Ward 2
42  District of Columbia  11003                   Ward 3
43  District of Columbia  11004                   Ward 4
44  District of Columbia  11005                   Ward 5
45  District of Columbia  11006                   Ward 6
46  District of Columbia  11007                   Ward 7
47  District of Columbia  11008                   Ward 8

In [40]:
# combine D.C. and Alaska legislative upper and lower house geography data with the Census geography data

ak_dc_census_geographies = pd.concat([census_geographies, ak_dc_geographies], ignore_index=True)

#sort by geoid
ak_dc_census_geographies = ak_dc_census_geographies.sort_values(by="geoid").reset_index(drop=True)

us_census_geographies = ak_dc_census_geographies.merge(
    all_census_states, 
    on='c_state_name', 
    how='left'
)

print(us_census_geographies.shape, '\n', us_census_geographies.dtypes)
us_census_geographies.head(100)

(3161, 5) 
 geoid            object
c_county_name    object
c_state_name     object
state_fips       object
state_abbr       object
dtype: object


geoid            c_county_name c_state_name state_fips state_abbr
0   01001           Autauga County      Alabama         01         AL
1   01003           Baldwin County      Alabama         01         AL
2   01005           Barbour County      Alabama         01         AL
3   01007              Bibb County      Alabama         01         AL
4   01009            Blount County      Alabama         01         AL
..    ...                      ...          ...        ...        ...
95  02029  State House District 29       Alaska         02         AK
96  02030  State House District 30       Alaska         02         AK
97  02031  State House District 31       Alaska         02         AK
98  02032  State House District 32       Alaska         02         AK
99  02033  State House District 33       Alaska         02         AK

[100 rows x 5 columns]

In [41]:
# get Alaska election results data from the Alaska Division of Elections

# Fetch the JSON data
url = "https://www.elections.alaska.gov/enr/results/statewide.js"
response = requests.get(url, verify=False)
data = response.json()

# Process candidate names
candidate_names_df = pd.DataFrame(data['candidateNames'], columns=['Candidate'])
candidate_names_df.reset_index(inplace=True)
candidate_names_df.rename(columns={'index': 'candidateNumber'}, inplace=True)
candidate_names_df = candidate_names_df.astype({"Candidate": str})
candidate_names_df = candidate_names_df[
    candidate_names_df['Candidate'].isin(["Harris/Walz (DEM)", "Trump/Vance (REP)"])
]

# Process contest names
contest_names_df = pd.DataFrame(data['contestNames'], columns=['Contest'])
contest_names_df.reset_index(inplace=True)
contest_names_df.rename(columns={'index': 'contestNumber'}, inplace=True)
contest_names_df = contest_names_df.astype({"Contest": str})
contest_names_df = contest_names_df[
    contest_names_df['Contest'] == "U.S. President / Vice President"
]

# Process districts
districts = data['districts']

# Flatten the JSON to extract required information
rows = []
for district in districts:
    house_district = district['number']
    for contests in district['contests']:
        contestNumber = contests['n']
        candidates = contests['c']
        for candidate in candidates:
            candidateNumber = candidate['n']
            votes = candidate['t']
            row = {
                    'House District': house_district, 
                    'contestNumber': contestNumber, 
                    'candidateNumber': candidateNumber,
                    'votes': votes
            }
            rows.append(row)

# Create the dataframe
cadidate_contests_df = pd.DataFrame(rows)

cadidate_contests_df['House District'] = "State House District " + cadidate_contests_df['House District'].astype(str)

cadidate_contests_df = cadidate_contests_df.astype({
    "candidateNumber": int,
    "contestNumber": int,
    "votes": int
})

# Merge contests with districts
districts_contests_df = pd.merge(
    cadidate_contests_df, contest_names_df,
    on="contestNumber", how="right"
)

# Merge candidates with districts-contests
districts_contests_candidates_df = pd.merge(
    districts_contests_df, candidate_names_df,
    left_on="candidateNumber", right_on="candidateNumber", how="right"
)

# Remove unnecessary columns
districts_contests_candidates_df.drop(columns=["contestNumber", "candidateNumber", "Contest"], inplace=True)

# Group by House District and calculate total votes
total_votes_df = districts_contests_candidates_df.groupby("House District", as_index=False).agg({"votes": "sum"}).rename(columns={"votes": "total_votes"})

# Pivot table for party votes
pivot_df = districts_contests_candidates_df.pivot_table(
    index="House District",
    columns="Candidate",
    values="votes",
    aggfunc="sum"
).reset_index(drop=False)
pivot_df.rename(columns={"Trump/Vance (REP)": "votes_gop", "Harris/Walz (DEM)": "votes_dem"}, inplace=True)

# Merge total votes with pivot table
ak_results_df = pd.merge(pivot_df, total_votes_df, on="House District", how="left")

# Merge with legislative districts
ak_results_geographies = pd.merge(
    ak_results_df, 
    ak_geographies,
    left_on="House District", 
    right_on="c_county_name", 
    how="left"
)

# Rename columns from the joined data
ak_results_geographies.rename(columns={"c_state_name": "state_name", "geoid": "county_fips", "c_county_name": "county_name"}, inplace=True)

# Reorder columns
ak_election_results = ak_results_geographies[[
    "state_name", "county_fips", "county_name", "votes_gop", "votes_dem", "total_votes"
]]

# # Reorder columns and remove unnecessary ones
# ak_election_results = ak_election_results[["state_name", "sldlst_name", "geoid", "total_votes", "votes_dem", "votes_gop"]]
# ak_election_results.rename(columns={"geoid": "county_fips", "sldlst_name": "county_name"}, inplace=True)

# Calculate GOP percentage
ak_election_results["per_gop"] = ak_election_results["votes_gop"] / ak_election_results["total_votes"]

# Calculate DEM percentage
ak_election_results["per_dem"] = ak_election_results["votes_dem"] / ak_election_results["total_votes"]

# Calculate vote difference
ak_election_results["diff"] = ak_election_results["votes_gop"] - ak_election_results["votes_dem"]

# Calculate percentage point difference
ak_election_results["per_point_diff"] = ak_election_results["per_gop"] - ak_election_results["per_dem"]

# Ensure column types are correctly set
ak_election_results["county_fips"] = ak_election_results["county_fips"].astype(str)
ak_election_results["total_votes"] = ak_election_results["total_votes"].astype("Int64")
ak_election_results["votes_dem"] = ak_election_results["votes_dem"].astype("Int64")
ak_election_results["votes_gop"] = ak_election_results["votes_gop"].astype("Int64")

# Sort by county_fips
ak_election_results = ak_election_results.sort_values(by="county_fips").reset_index(drop=True)

print(ak_election_results.shape, '\n', ak_election_results.dtypes)
ak_election_results.head(100)

(40, 10) 
 state_name         object
county_fips        object
county_name        object
votes_gop           Int64
votes_dem           Int64
total_votes         Int64
per_gop           float64
per_dem           float64
diff                int64
per_point_diff    float64
dtype: object


state_name county_fips              county_name  votes_gop  votes_dem  \
0      Alaska       02001   State House District 1       4861       3367   
1      Alaska       02002   State House District 2       4534       4571   
2      Alaska       02003   State House District 3       4496       6130   
3      Alaska       02004   State House District 4       2691       6161   
4      Alaska       02005   State House District 5       4351       3328   
5      Alaska       02006   State House District 6       6705       4773   
6      Alaska       02007   State House District 7       6777       2513   
7      Alaska       02008   State House District 8       7950       2576   
8      Alaska       02009   State House District 9       5581       5849   
9      Alaska       02010  State House District 10       4357       4070   
10     Alaska       02011  State House District 11       4864       4633   
11     Alaska       02012  State House District 12       4059       4169   
12     Alaska       02013  State House District 13       3589       3682   
13     Alaska       02014  State House District 14       2768       4346   
14     Alaska       02015  State House District 15       4594       4364   
15     Alaska       02016  State House District 16       3929       5344   
16     Alaska       02017  State House District 17       2407       4990   
17     Alaska       02018  State House District 18       2686       1928   
18     Alaska       02019  State House District 19       1971       2825   
19     Alaska       02020  State House District 20       3041       3924   
20     Alaska       02021  State House District 21       3984       4711   
21     Alaska       02022  State House District 22       2594       2660   
22     Alaska       02023  State House District 23       6101       4116   
23     Alaska       02024  State House District 24       6482       3234   
24     Alaska       02025  State House District 25       6891       3172   
25     Alaska       02026  State House District 26       7143       2013   
26     Alaska       02027  State House District 27       6785       1807   
27     Alaska       02028  State House District 28       6713       2213   
28     Alaska       02029  State House District 29       7418       2708   
29     Alaska       02030  State House District 30       7312       2701   
30     Alaska       02031  State House District 31       3475       3240   
31     Alaska       02032  State House District 32       3742       1883   
32     Alaska       02033  State House District 33       6548       2016   
33     Alaska       02034  State House District 34       5836       3759   
34     Alaska       02035  State House District 35       4395       4885   
35     Alaska       02036  State House District 36       5633       3538   
36     Alaska       02037  State House District 37       2208       1921   
37     Alaska       02038  State House District 38       1558       2028   
38     Alaska       02039  State House District 39       1690       2180   
39     Alaska       02040  State House District 40       1688       1362   

    total_votes   per_gop   per_dem  diff  per_point_diff  
0          8228  0.590788  0.409212  1494        0.181575  
1          9105  0.497968  0.502032   -37       -0.004064  
2         10626  0.423113  0.576887 -1634       -0.153774  
3          8852  0.303999  0.696001 -3470       -0.392002  
4          7679  0.566610  0.433390  1023        0.133220  
5         11478  0.584161  0.415839  1932        0.168322  
6          9290  0.729494  0.270506  4264        0.458988  
7         10526  0.755273  0.244727  5374        0.510545  
8         11430  0.488276  0.511724  -268       -0.023447  
9          8427  0.517029  0.482971   287        0.034057  
10         9497  0.512162  0.487838   231        0.024323  
11         8228  0.493316  0.506684  -110       -0.013369  
12         7271  0.493605  0.506395   -93       -0.012791  
13         7114  0.389092  0.610908 -1578     

In [42]:
# get Wshington D.C. election results data from the District of Columbia Board of Elections

# extract the ward numbers from the c_county_name column
wards = dc_geographies.c_county_name.str.extract(r'Ward (\d+)')[0].astype(str)
# census_gaz_counties[~census_gaz_counties.county_file_name.str.extract(r'_(\d{2})\.txt')[0].isin(suzerainty_fips)]

# Fetch the JSON data
def get_ward_election_results(ward_number):
    # Fetch the JSON data from the API
    # url = f"https://electionresults.dcboe.org/ward/getWard/2024-General-Election/1"
    url = f"https://electionresults.dcboe.org/ward/getWard/2024-General-Election/{ward_number}"
    response = requests.get(url)
    data = response.json()

    # Convert JSON data to DataFrame
    dc_records_df = pd.DataFrame(data)

    # Filter rows where OfficeName is "ELECTORS OF PRESIDENT AND VICE PRESIDENT OF THE UNITED STATES"
    dc_president_results_df = dc_records_df[dc_records_df["OfficeName"] == "ELECTORS OF PRESIDENT AND VICE PRESIDENT OF THE UNITED STATES"]

    dc_candidates = pd.json_normalize(dc_president_results_df['ElectionData'])

    # Flatten the JSON to extract required information
    rows = []
    for candidate in dc_candidates.iloc[0]:
        votes = candidate['Votes']
        total_votes = candidate['TotalVotes']
        party_code = candidate['PartyCode']
        row = {
            'WardNumber': ward_number,
            'PartyCode': party_code,
            'votes': votes,
            'total_votes': total_votes
        }
        rows.append(row)

    # Convert rows to DataFrame
    dc_candidates_df = pd.DataFrame(rows)

    # Filter for DEM and REP candidates
    dc_candidates_df = dc_candidates_df[dc_candidates_df['PartyCode'].isin(['DEM', 'REP'])]

    # Pivot the data to separate GOP and DEM votes into their own columns
    dc_pivot_df = dc_candidates_df.pivot_table(
        index=["WardNumber", "total_votes"],
        columns="PartyCode",
        values="votes",
        fill_value=0
    )
    dc_pivot_df.reset_index(inplace=True)

    dc_pivot_df.columns.name = None  # Remove the name of the 'PartyCode' index
    dc_pivot_df.rename(columns={"WardNumber": "Ward", "REP": "votes_gop", "DEM": "votes_dem"}, inplace=True)

    dc_ward_results = dc_pivot_df[["Ward", "votes_gop", "votes_dem", "total_votes"]]

    # Add the term "Ward " to the Ward column
    dc_ward_results["Ward"] = "Ward " + dc_ward_results["Ward"].astype(str)

    return dc_ward_results

# Retrieve results for each ward
dc_results = [get_ward_election_results(ward_number) for ward_number in wards]

# Combine all ward dataframes
dc_results_df = pd.concat(dc_results, ignore_index=True)

# Merge with legislative districts
dc_results_geographies = pd.merge(
    dc_results_df, 
    dc_geographies,
    left_on="Ward", 
    right_on="c_county_name", 
    how="left"
)

# Rename columns from the joined data
dc_results_geographies.rename(columns={"c_state_name": "state_name", "geoid": "county_fips", "c_county_name": "county_name"}, inplace=True)

# Reorder columns
dc_election_results = dc_results_geographies[[
    "state_name", "county_fips", "county_name", "votes_gop", "votes_dem", "total_votes"
]]

# Calculate GOP percentage
dc_election_results["per_gop"] = dc_election_results["votes_gop"] / dc_election_results["total_votes"]

# Calculate DEM percentage
dc_election_results["per_dem"] = dc_election_results["votes_dem"] / dc_election_results["total_votes"]

# Calculate vote difference
dc_election_results["diff"] = dc_election_results["votes_gop"] - dc_election_results["votes_dem"]

# Calculate percentage point difference
dc_election_results["per_point_diff"] = dc_election_results["per_gop"] - dc_election_results["per_dem"]

# Ensure column types are correctly set
dc_election_results["county_fips"] = dc_election_results["county_fips"].astype(str)
dc_election_results["total_votes"] = dc_election_results["total_votes"].astype("Int64")
dc_election_results["votes_dem"] = dc_election_results["votes_dem"].astype("Int64")
dc_election_results["votes_gop"] = dc_election_results["votes_gop"].astype("Int64")

# Sort by county_fips
dc_election_results = dc_election_results.sort_values(by="county_fips").reset_index(drop=True)

print(dc_election_results.shape, '\n', dc_election_results.dtypes)
dc_election_results.head(100)

(8, 10) 
 state_name         object
county_fips        object
county_name        object
votes_gop           Int64
votes_dem           Int64
total_votes         Int64
per_gop           float64
per_dem           float64
diff              float64
per_point_diff    float64
dtype: object


state_name county_fips county_name  votes_gop  votes_dem  \
0  District of Columbia       11001      Ward 1       2056      36882   
1  District of Columbia       11002      Ward 2       2982      33571   
2  District of Columbia       11003      Ward 3       3544      39521   
3  District of Columbia       11004      Ward 4       2511      38628   
4  District of Columbia       11005      Ward 5       2309      41413   
5  District of Columbia       11006      Ward 6       3638      43262   
6  District of Columbia       11007      Ward 7       2092      33972   
7  District of Columbia       11008      Ward 8       1944      26936   

   total_votes   per_gop   per_dem     diff  per_point_diff  
0        40463  0.050812  0.911499 -34826.0       -0.860688  
1        37959  0.078558  0.884402 -30589.0       -0.805843  
2        44584  0.079490  0.886439 -35977.0       -0.806949  
3        42403  0.059218  0.910973 -36117.0       -0.851756  
4        45114  0.051181  0.917963 -39104.0       -0.866782  
5        48678  0.074736  0.888738 -39624.0       -0.814002  
6        37037  0.056484  0.917245 -31880.0       -0.860761  
7        29631  0.065607  0.909048 -24992.0       -0.843441

In [43]:
# combine D.C. and Alaska election results with election results from the contiguous U.S.
all_election_results = pd.concat([election_results, dc_election_results, ak_election_results], ignore_index=True)

# Sort by geoid
all_election_results = all_election_results.sort_values(by="county_fips").reset_index(drop=True)

print(all_election_results.shape, '\n', all_election_results.dtypes)
all_election_results.head(100)

(3160, 10) 
 state_name         object
county_fips        object
county_name        object
votes_gop           Int64
votes_dem           Int64
total_votes         Int64
diff              float64
per_gop           float64
per_dem           float64
per_point_diff    float64
dtype: object


state_name county_fips              county_name  votes_gop  votes_dem  \
0     Alabama       01001           Autauga County      20484       7439   
1     Alabama       01003           Baldwin County      95798      24934   
2     Alabama       01005           Barbour County       5606       4158   
3     Alabama       01007              Bibb County       7572       1619   
4     Alabama       01009            Blount County      25354       2576   
..        ...         ...                      ...        ...        ...   
95     Alaska       02029  State House District 29       7418       2708   
96     Alaska       02030  State House District 30       7312       2701   
97     Alaska       02031  State House District 31       3475       3240   
98     Alaska       02032  State House District 32       3742       1883   
99     Alaska       02033  State House District 33       6548       2016   

    total_votes     diff   per_gop   per_dem  per_point_diff  
0         28190  13045.0  0.726641  0.263888        0.462753  
1        121808  70864.0  0.786467  0.204699        0.581768  
2          9832   1448.0  0.570179  0.422905        0.147274  
3          9241   5953.0  0.819392  0.175197        0.644194  
4         28115  22778.0  0.901796  0.091624        0.810173  
..          ...      ...       ...       ...             ...  
95        10126   4710.0  0.732570  0.267430        0.465139  
96        10013   4611.0  0.730251  0.269749        0.460501  
97         6715    235.0  0.517498  0.482502        0.034996  
98         5625   1859.0  0.665244  0.334756        0.330489  
99         8564   4532.0  0.764596  0.235404        0.529192  

[100 rows x 10 columns]

In [44]:
# export election results to a CSV file

# specify the file path for the CSV export
file_path = "2024_US_County_Level_Presidential_Results.csv"

# Export the DataFrame to a CSV file
all_election_results.to_csv(file_path, index=False)

print(f"DataFrame exported to {file_path}")

DataFrame exported to 2024_US_County_Level_Presidential_Results.csv
